In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import utility
import math
from time import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.metrics import classification_report

In [ ]:
#SklearnDatasets
dataset = datasets.load_breast_cancer() # load_iris, load_wine, load_breast_cancer()
df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
dir_path = 'Breast_Cancer'

In [ ]:
scaler = MinMaxScaler()
scaler.fit(dataset.data)
scaled_df = scaler.transform(dataset.data)
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)

In [ ]:
df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
targets = dataset.target
df_scaled['target'] = targets
num_classes = len(df_scaled['target'].unique())
display(df_scaled)

In [ ]:
np.random.seed(50)
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
n_neurons = X_train.shape[1]
n_hidden_layers = 1
print(f'n_hidden_layers: {n_hidden_layers}, n_neurons {n_neurons}')
model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),  # Explicit input layer
    layers.Dense(n_neurons, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model_path = f'new_models/{dir_path}.keras'
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
ck = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)
start = time()
model.fit(X_train,y_train, epochs=400, batch_size=32,validation_data=(X_test, y_test), verbose=1, callbacks=[ck, es])
print(f'Training time: {time()-start}')
model = tf.keras.models.load_model(model_path)
print('Training Results')
model.evaluate(X_train, y_train, verbose=2)
print('Test Results')
model.evaluate(X_test, y_test, verbose=2)
model.save(f'new_models/{dir_path}.h5', save_format='h5')

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

y_test_classes = np.argmax(y_test, axis=1)

report = classification_report(y_test_classes, y_pred_classes,digits=4)
print(report)

# Save the report to a text file
with open(f'new_models/{dir_path}_report.txt', 'w') as f:
    f.write(report)